In [0]:
dbutils.widgets.text("data_execucao", "")
data_execucao = dbutils.widgets.get("data_execucao")

In [0]:
import requests
from pyspark.sql.functions import to_date, year, month, dayofmonth
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import requests
from pyspark.sql.types import StructType, StructField, StringType, DateType, DoubleType


In [0]:
import requests
import time
import datetime
import pandas as pd

def get_closing_price(coin_id, date_str, vs_currency="usd"):
    date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    start_ts = int(time.mktime(date_obj.timetuple()))
    end_ts = int(time.mktime((date_obj + datetime.timedelta(days=1)).timetuple()))

    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
    params = {
        "vs_currency": vs_currency,
        "from": start_ts,
        "to": end_ts
    }

    r = requests.get(url, params=params)
    data = r.json()

    return data["prices"][-1][1]

In [0]:
def get_multiple_closings(coin_list, date_str, vs_currency="usd"):
    results = []
    for coin in coin_list:
        price = get_closing_price(coin, date_str, vs_currency)
        results.append({
            "moeda": coin,
            "data": date_str,
            "fechamento": price
        })
        time.sleep(1)

    return results

In [0]:
coins = [
    "bitcoin", "ethereum", "solana", "binancecoin", "ripple", "dogecoin", "tron", "cardano"
]


results = get_multiple_closings(coins, data_execucao)

In [0]:
schema_tmp = StructType([
    StructField("moeda", StringType(), False),
    StructField("data",  StringType(),  False),   # <- string por enquanto
    StructField("fechamento", DoubleType(), True),
])

df = (
    spark.createDataFrame(results, schema=schema_tmp)
        .withColumn("data", F.to_date("data", "yyyy-MM-dd"))  
        .withColumn("fechamento", F.col("fechamento").cast("double"))
        .withColumn("ano", F.year("data"))
        .withColumn("mes", F.month("data"))
        .withColumn("dia", F.dayofmonth("data"))
)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS airflow_databricks.default.bronze (
  moeda STRING,
  data DATE,
  fechamento DOUBLE,
  ano INT,
  mes INT,
  dia INT
)
USING DELTA
PARTITIONED BY (ano, mes, dia)

In [0]:
df.write\
    .format("delta")\
    .mode("append")\
    .insertInto("airflow_databricks.default.bronze")